In [1]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import numpy as np

from SVR import SVR
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae
from sklearn import svm, preprocessing

from utils import plot_error, plot_sv_number

np.random.seed(42)
n_features = 10
X, y = make_regression(n_samples=200, n_features=n_features)

X = preprocessing.StandardScaler().fit(X).transform(X)
y = 2*(y-min(y))/(max(y)-min(y)) - 1

print(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

C=1
kernel = 'poly'

model = SVR(C=C, kernel=kernel)
train_err = []
test_err = []

sv = []
sv_sota = []

model_sota = svm.SVR(C=C, kernel=kernel)
train_err_sota = []
test_err_sota = []

batch_size = int(len(X_train) / 10)

print(batch_size)

for i in range(0, int(len(X)/batch_size)):
    print("i={}----------------------------------------------------------------------".format(i))
    bs = (i+1)*batch_size

    n_sv, alphas, indices = model.train(X_train[:bs], y_train[:bs], sigma = 1 / (n_features * X_train.var()))
    prediction = model.predict(X_train)

    train_err.append(mse(prediction, y_train))
    test_err.append(mse(model.predict(X_test), y_test))

    model_sota.fit(X_train[:bs], y_train[:bs])
    prediction = model_sota.predict(X_train)
    # support_vector_indices = np.where(np.abs(model_sota.decision_function(X_train[:bs])) <= 1 + 1e-6)[0]
    # support_vectors = (X_train[:bs])[support_vector_indices]
    # n_sv_sota = len(support_vectors)

    train_err_sota.append(mse(prediction, y_train))
    test_err_sota.append(mse(model_sota.predict(X_test), y_test))

    # sv.append(n_sv)
    # sv_sota.append(n_sv_sota)
    # print("data: {}, support_vectors: {}, smallest: {}, greatest: {},".format(bs, n_sv, min(alphas), max(alphas), ))
    # print("data: {}, support_vectors: {}, smallest: {}, greatest: {}, ".format(bs, n_sv_sota, min(alphas), max(alphas), ))

plot_error(train_err, test_err, "mySVM")
plot_error(train_err_sota, test_err_sota, "sklearn")
# plot_sv_number(sv)
# plot_sv_number(sv_sota)

[-0.22425941  0.17619192  0.56721442 -0.27211054  0.64453396  0.27915316
 -0.09938309  0.09010784  1.          0.01165163 -0.06949095 -0.63355079
  0.51362916  0.07686534  0.68883378  0.29654706 -0.21246623 -0.10643141
 -0.20079187  0.03968299  0.15258515  0.85433956 -0.30009764  0.07255097
 -0.2928296  -0.78033686 -0.05220127  0.56977038  0.67822511 -0.50161891
  0.00660735 -0.48330955 -0.13008954  0.18591618  0.01635973  0.02340874
  0.25457331  0.31603815 -0.00886788  0.1080848   0.5195438   0.43558753
 -0.48939521  0.31540898  0.47244562  0.10080848  0.00221507  0.07780556
  0.60809752 -0.50675326  0.2157999   0.03114202  0.17325208  0.27252942
 -0.4939009   0.05203743  0.78671977 -0.08063017 -0.60703924  0.15628502
 -0.17269367  0.47628742  0.22500009 -0.35676415 -0.259283   -0.08032093
 -0.11793882  0.34770483  0.65804627  0.55211036  0.18517845  0.04293944
  0.06359631  0.13534276  0.09946244 -0.12724209  0.2008744  -0.01966381
  0.25936249 -0.05837741  0.26912672  0.38871197 -0

KeyboardInterrupt: 